# Testing logratio transformations

In [27]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import numpy as np
import sys

sys.path.insert(0, "..")
from eis_toolkit.transformations.coda.alr import _replace_zeros_with_nan_inplace, ALR_transform, _ALR_transform
from eis_toolkit.transformations.coda.clr import CLR_transform, inverse_CLR
from eis_toolkit.transformations.coda.ilr import single_ILR_transform
from eis_toolkit.transformations.coda.pairwise import single_pairwise_logratio
from eis_toolkit.transformations.coda.plr import PLR_transform, single_PLR_transform

GEOCHEMICAL_DATA = "../tests/data/local/coda/IOCG_CLB_Till_Geochem_reg_511p.shp"

In [28]:
# Change option to display all columns (the geochemical data in question has a lot of columns)

pd.set_option('display.max_columns', None)

## Testing with a simple, single-row composition

In [29]:
# A simple example composition consisiting of the parts a, b and c.
# Components are expressed in percentages, and the parts sum to 100%.
# The example is from Pawlowsky-Glahn & Egozcue 2006.

c_arr = np.array([80, 15, 5])
C = pd.DataFrame(c_arr[None], columns=["a", "b", "c"])

In [30]:
p_a_b = single_pairwise_logratio(float(C.iloc[0, 0]), float(C.iloc[0, 1]))
p_a_c = single_pairwise_logratio(float(C.iloc[0, 0]), float(C.iloc[0, 2]))
p_b_c = single_pairwise_logratio(float(C.iloc[0, 1]), float(C.iloc[0, 2]))

C_alr = ALR_transform(C)
C_clr = CLR_transform(C)
C_plr = PLR_transform(C)

C_clr_inv, _ = inverse_CLR(C_clr)

C_ilr_ab = single_ILR_transform(C, ["a"], ["b"])
C_ilr_ab_c = single_ILR_transform(C, ["a", "b"], ["c"])

In [31]:
# Expected result: 1.67
p_a_b

1.6739764335716716

In [32]:
# Expected result: 2.77
p_a_c

2.772588722239781

In [33]:
# Expected result: 1.10
p_b_c

1.0986122886681098

In [34]:
# Expected result: [2.77, 1.10]
C_alr

,a,b
0,2.772589,1.098612


In [35]:
# Expected result: [1.48, -0.19, -1.29]
C_clr

,a,b,c
0,1.482188,-0.191788,-1.2904


In [36]:
# Expected result: [1.82, 0.78]
C_plr

,a,b
0,1.815303,0.776836


In [11]:
# Renormalized to 100%, CLR inverse should show the original data: [80, 15, 5]
C_clr_inv * 100

,a,b,c
0,80.0,15.0,5.0


In [37]:
# Expected result: 1.18
C_ilr_ab

0    1.18368
dtype: float64

In [38]:
# Expected result: 1.58
C_ilr_ab_c

0    1.580411
dtype: float64

### Testing with example data

In [59]:
# Define some constants

ppm = 1e-6
million = 1e6

In [60]:
# Names of all the columns that contain concentration data

elements = ['Al_ppm_511', 'Ba_ppm_511', 'Ca_ppm_511', 'Co_ppm_511', 'Cr_ppm_511', 'Cu_ppm_511', 'Fe_ppm_511', 'K_ppm_511P', 'La_ppm_511', 'Li_ppm_511', 'Mg_ppm_511', 'Mn_ppm_511', 'Ni_ppm_511', 'P_ppm_511P', 'Sc_ppm_511', 'Sr_ppm_511', 'Th_ppm_511', 'Ti_ppm_511', 'V_ppm_511P', 'Y_ppm_511P', 'Zn_ppm_511', 'Zr_ppm_511']

In [61]:
# A subcomposition (selected at random)

elements_to_analyze = ['Al_ppm_511', 'Ca_ppm_511', 'Fe_ppm_511', 'Mg_ppm_511']

In [55]:
df_all = gpd.read_file(GEOCHEMICAL_DATA)
#df_all = pd.DataFrame(df.drop(columns='geometry'))
df_all.head()

,OBJECTID,Year,Observatio,Northing,Easting,Northing_k,Easting_kk,Map_sheet_,Map_sheet1,Material,Soil_horiz,Sampler,Parallel_s,Combined_s,Fraction,Second_ana,Analysis_d,Al_ppm_511,Al_511P_T,Ba_ppm_511,Ba_511P_T,Ca_ppm_511,Ca_511P_T,Co_ppm_511,Co_511P_T,Cr_ppm_511,Cr_511P_T,Cu_ppm_511,Cu_511P_T,Fe_ppm_511,Fe_511P_T,K_ppm_511P,K_511P_T,La_ppm_511,La_511P_T,Li_ppm_511,Li_511P_T,Mg_ppm_511,Mg_511P_T,Mn_ppm_511,Mn_511P_T,Ni_ppm_511,Ni_511P_T,P_ppm_511P,P_511P_T,Sc_ppm_511,Sc_511P_T,Sr_ppm_511,Sr_511P_T,Th_ppm_511,Th_511P_T,Ti_ppm_511,Ti_511P_T,V_ppm_511P,V_511P_T,Y_ppm_511P,Y_511P_T,Zn_ppm_511,Zn_511P_T,Zr_ppm_511,Zr_511P_T,geometry
0,2259,81,49114,7526963,404203,7530108,3404337,2741,10,till,C,percussion drilling/Cobra,None,Y,"sieved, <0.06 mm (fine)",None,1986-07-01,27600.0,None,20.6,None,40200.0,None,69.70,None,73.4,None,100.0,None,83200.0,None,664.0,None,11.3,None,14.30,None,17200.0,None,2250.0,None,61.4,None,735.0,None,24.30,None,22.60,None,2.97,None,2590.0,None,354.0,None,12.20,None,86.7,None,0.0,<,POINT (404203.131 7526963.173)
1,3328,81,49282,7530498,406827,7533644,3406962,2741,11,till,C,percussion drilling/Cobra,None,Y,"sieved, <0.06 mm (fine)",None,1986-07-24,14100.0,None,22.6,None,5000.0,None,20.50,None,41.0,None,60.8,None,28300.0,None,405.0,None,10.8,None,8.16,None,7520.0,None,370.0,None,34.2,None,566.0,None,5.70,None,12.00,None,1.74,None,2640.0,None,76.9,None,9.00,None,58.5,None,0.0,<,POINT (406827.101 7530497.757)
2,3507,81,49269,7536878,406536,7540027,3406671,2741,11,till,C,percussion drilling/Cobra,None,Y,"sieved, <0.06 mm (fine)",None,1986-06-24,7880.0,None,22.9,None,3070.0,None,6.94,None,33.0,None,24.7,None,14500.0,None,710.0,None,11.7,None,7.58,None,4540.0,None,152.0,None,17.9,None,458.0,None,3.12,None,8.36,None,5.44,None,1470.0,None,46.3,None,6.90,None,28.7,None,12.3,None,POINT (406536.240 7536878.222)
3,4936,81,49245,7524138,401544,7527282,3401677,2741,07,till,C,percussion drilling/Cobra,None,Y,"sieved, <0.06 mm (fine)",None,1986-06-12,7300.0,None,25.1,None,3290.0,None,8.38,None,25.0,None,28.4,None,14600.0,None,836.0,None,10.6,None,5.62,None,3240.0,None,156.0,None,13.8,None,744.0,None,2.28,None,6.90,None,4.42,None,1050.0,None,42.2,None,5.03,None,16.7,None,7.9,None,POINT (401544.178 7524138.307)
4,4937,81,49283,7530830,405584,7533976,3405718,2741,11,till,C,percussion drilling/Cobra,None,Y,"sieved, <0.06 mm (fine)",None,1986-07-10,12500.0,None,25.1,None,3600.0,None,29.30,None,38.9,None,88.7,None,31500.0,None,1260.0,None,10.9,None,6.66,None,8020.0,None,392.0,None,36.4,None,592.0,None,4.17,None,8.25,None,2.58,None,1530.0,None,69.7,None,7.74,None,60.6,None,0.0,<,POINT (405583.597 7530829.630)


In [66]:
# (Testing)
# A function for checking if zeros should be considered to be rounded

def contains_values_below_detection_limit(series):
    return "<" in series.values

contains_values_below_detection_limit(df_all["Zr_511P_T"])
contains_values_below_detection_limit(df_all["Y_511P_T"])

False

In [40]:
# Read the vector file into a dataframe

df = gpd.read_file(GEOCHEMICAL_DATA, include_fields=elements)
df = pd.DataFrame(df.drop(columns='geometry'))

# TODO: add a column for the residual

df["sum"] = np.nansum(df)
print(df["sum"][0])

#_replace_zeros_with_nan_inplace(df)
#df.dropna()
df.head()

#df.iloc[0,:]

60591619.90400001


,Al_ppm_511,Ba_ppm_511,Ca_ppm_511,Co_ppm_511,Cr_ppm_511,Cu_ppm_511,Fe_ppm_511,K_ppm_511P,La_ppm_511,Li_ppm_511,Mg_ppm_511,Mn_ppm_511,Ni_ppm_511,P_ppm_511P,Sc_ppm_511,Sr_ppm_511,Th_ppm_511,Ti_ppm_511,V_ppm_511P,Y_ppm_511P,Zn_ppm_511,Zr_ppm_511,sum
0,27600.0,20.6,40200.0,69.70,73.4,100.0,83200.0,664.0,11.3,14.30,17200.0,2250.0,61.4,735.0,24.30,22.60,2.97,2590.0,354.0,12.20,86.7,0.0,6.059162e+07
1,14100.0,22.6,5000.0,20.50,41.0,60.8,28300.0,405.0,10.8,8.16,7520.0,370.0,34.2,566.0,5.70,12.00,1.74,2640.0,76.9,9.00,58.5,0.0,6.059162e+07
2,7880.0,22.9,3070.0,6.94,33.0,24.7,14500.0,710.0,11.7,7.58,4540.0,152.0,17.9,458.0,3.12,8.36,5.44,1470.0,46.3,6.90,28.7,12.3,6.059162e+07
3,7300.0,25.1,3290.0,8.38,25.0,28.4,14600.0,836.0,10.6,5.62,3240.0,156.0,13.8,744.0,2.28,6.90,4.42,1050.0,42.2,5.03,16.7,7.9,6.059162e+07
4,12500.0,25.1,3600.0,29.30,38.9,88.7,31500.0,1260.0,10.9,6.66,8020.0,392.0,36.4,592.0,4.17,8.25,2.58,1530.0,69.7,7.74,60.6,0.0,6.059162e+07


In [53]:
# Check if the sum of each composition is the same

def all_values_same(series):
    arr = series.to_numpy()
    return (arr[0] == arr).all()

all_values_same(df["sum"])

True

In [51]:
df.describe().loc["mean",:]

Al_ppm_511    1.379913e+04
Ba_ppm_511    7.498031e+01
Ca_ppm_511    3.700737e+03
Co_ppm_511    1.226708e+01
Cr_ppm_511    5.402185e+01
Cu_ppm_511    4.358362e+01
Fe_ppm_511    2.484711e+04
K_ppm_511P    2.198773e+03
La_ppm_511    2.629311e+01
Li_ppm_511    1.198001e+01
Mg_ppm_511    6.560192e+03
Mn_ppm_511    2.628523e+02
Ni_ppm_511    3.002676e+01
P_ppm_511P    8.609685e+02
Sc_ppm_511    3.952198e+00
Sr_ppm_511    1.316853e+01
Th_ppm_511    1.011247e+01
Ti_ppm_511    1.859682e+03
V_ppm_511P    6.610621e+01
Y_ppm_511P    1.099033e+01
Zn_ppm_511    2.998996e+01
Zr_ppm_511    1.194834e+01
sum           6.059162e+07
Name: mean, dtype: float64

In [50]:
np.sum(df.describe().loc["mean",:])

60646108.77082014

In [44]:
df*0.016503932391971034

,Al_ppm_511,Ba_ppm_511,Ca_ppm_511,Co_ppm_511,Cr_ppm_511,Cu_ppm_511,Fe_ppm_511,K_ppm_511P,La_ppm_511,Li_ppm_511,Mg_ppm_511,Mn_ppm_511,Ni_ppm_511,P_ppm_511P,Sc_ppm_511,Sr_ppm_511,Th_ppm_511,Ti_ppm_511,V_ppm_511P,Y_ppm_511P,Zn_ppm_511,Zr_ppm_511,sum
0,455.508534,0.339981,663.458082,1.150324,1.211389,1.650393,1373.127175,10.958611,0.186494,0.236006,283.867637,37.133848,1.013341,12.130390,0.401046,0.372989,0.049017,42.745185,5.842392,0.201348,1.430891,0.000000,999999.998416
1,232.705447,0.372989,82.519662,0.338331,0.676661,1.003439,467.061287,6.684093,0.178242,0.134672,124.109572,6.106455,0.564434,9.341226,0.094072,0.198047,0.028717,43.570382,1.269152,0.148535,0.965480,0.000000,999999.998416
2,130.050987,0.377940,50.667072,0.114537,0.544630,0.407647,239.307020,11.717792,0.193096,0.125100,74.927853,2.508598,0.295420,7.558801,0.051492,0.137973,0.089781,24.260781,0.764132,0.113877,0.473663,0.202998,999999.998416
3,120.478706,0.414249,54.297938,0.138303,0.412598,0.468712,240.957413,13.797287,0.174942,0.092752,53.472741,2.574613,0.227754,12.278926,0.037629,0.113877,0.072947,17.329129,0.696466,0.083015,0.275616,0.130381,999999.998416
4,206.299155,0.414249,59.414157,0.483565,0.642003,1.463899,519.873870,20.794955,0.179893,0.109916,132.361538,6.469541,0.600743,9.770328,0.068821,0.136157,0.042580,25.251017,1.150324,0.127740,1.000138,0.000000,999999.998416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,306.973142,4.934676,191.445616,0.264063,0.783937,1.255949,648.604543,64.530376,0.849953,0.371338,145.069566,4.753133,0.465411,45.715893,0.101499,0.541329,0.163884,60.899511,1.980472,0.311924,0.932472,0.344932,999999.998416
1108,551.231342,10.958611,39.444398,0.698116,0.534727,1.183332,1145.372908,163.058852,0.252510,0.358135,305.322749,17.824247,0.721222,9.819840,0.250860,0.098363,0.057434,63.375100,3.036724,0.202998,0.496768,0.476964,999999.998416
1109,632.100611,11.618768,67.171005,0.797140,1.897952,2.607621,1280.705154,121.633982,0.694816,0.434053,452.207748,18.484404,1.589329,16.999050,0.055453,0.173291,0.057434,47.201247,2.756157,0.269014,5.974424,0.000000,999999.998416
1110,538.028196,16.157350,142.428937,1.028195,0.559483,9.258706,1143.722515,226.103874,0.546280,1.051300,308.623536,28.881882,1.599231,21.950230,0.056939,0.202998,0.041590,71.627067,2.558110,0.270664,11.321698,0.643653,999999.998416


In [43]:
1000000/60591620.0

0.016503932391971034

In [19]:
alr_transformed = _ALR_transform(df, ['Al_ppm_511', 'Ca_ppm_511', 'Fe_ppm_511'], -2, True)

In [20]:
np.log(df.iloc[2,0]/df.iloc[2,3])

0.5514008918165313

In [21]:
alr_transformed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Al_ppm_511  1112 non-null   float64
 1   Ca_ppm_511  1112 non-null   float64
 2   Fe_ppm_511  1112 non-null   float64
dtypes: float64(3)
memory usage: 26.2 KB


In [22]:
alr_transformed

,Al_ppm_511,Ca_ppm_511,Fe_ppm_511
0,-1.103432,-0.727380,0.0
1,-0.696687,-1.733424,0.0
2,-0.609821,-1.552471,0.0
3,-0.693147,-1.490134,0.0
4,-0.924259,-2.169054,0.0
...,...,...,...
1107,-0.748063,-1.220219,0.0
1108,-0.731331,-3.368594,0.0
1109,-0.706118,-2.947924,0.0
1110,-0.754133,-2.083200,0.0
